In [1]:
import cv2
import tensorflow as tf

from glob import glob
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import imgaug.augmenters as iaa
from imgaug import parameters as iap

In [2]:
# Define your dataset paths
rgb_img_paths = sorted(
    list(
        itertools.chain.from_iterable(
            [glob(i + "*.jpg") for i in glob("vignettes/rgb/*/", recursive = True)]
        )
    )
)
masks_img_paths = sorted(
    list(
        itertools.chain.from_iterable(
            [glob(i + "*.png") for i in glob("vignettes/mask/*/", recursive = True)]
        )
    )
)

In [3]:
rgb_img_paths

['vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780000.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780144.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780288.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780432.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780576.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780720.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780000.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780144.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780288.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780432.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780576.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305144_6780720.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305288_6780000.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305288_6780144.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305288_6780288.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305288_6780432.jpg',
 'vignettes/rgb\\rgb_305_6780\\rgb_35_305288_6780576.jpg

In [4]:
# Function to preprocess an image with histogram equalization
def preprocess_image(image_path, mask_path):
    # Read an image from a file
    image_string = tf.io.read_file(image_path)
    mask_string = tf.io.read_file(mask_path)
    # print(image_string, mask_string)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_equalized = tf.numpy_function(equalize_histogram, [image_decoded], tf.float32)
    
    # Normalize the image
    image_output = (image_equalized - tf.reduce_min(image_equalized)) / (tf.reduce_max(image_equalized) - tf.reduce_min(image_equalized))
    
    mask_decoded = tf.image.decode_jpeg(mask_string, channels=1)
    
    # Ensure mask_output is of type float32
    mask_output = tf.cast(mask_decoded, tf.float32)
    
    return (image_output, mask_output)


# Function to apply histogram equalization to an image
def equalize_histogram(image):
    # Convert RGB to YUV
    image_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    # Apply histogram equalization to the Y channel
    image_yuv[:,:,0] = cv2.equalizeHist(image_yuv[:,:,0])
    # Convert YUV back to RGB
    image_output = cv2.cvtColor(image_yuv, cv2.COLOR_YUV2RGB)
    image_output = tf.convert_to_tensor(image_output, dtype=tf.float32)
    return image_output

# # Load your image and mask data
# image_path = 'vignettes/rgb\\rgb_305_6780\\rgb_35_305000_6780000.jpg'# 'vignettes/rgb/rgb_305_6785/rgb_35_305000_6781000.jpg'
# mask_path = 'vignettes/mask\\mask_305_6780\\mask_35_305000_6780000.png'

# # Preprocess the image and mask
# image, mask = preprocess_image(image_path, mask_path)

In [5]:
BATCH_SIZE = 32 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations
CHANNELS = 3
IMG_SIZE = 256

In [6]:
# Create a first dataset of file paths and labels
dataset = tf.data.Dataset.from_tensor_slices((rgb_img_paths, masks_img_paths))

In [7]:
dataset = dataset.map(preprocess_image, num_parallel_calls=AUTOTUNE)

In [8]:
elems = list(dataset.as_numpy_iterator())

In [17]:
def gaussian_blur(image, kernel_size=(3, 3)):
    image = cv2.GaussianBlur(image, kernel_size, 0)
    return image
    
def data_augmentation(image, mask):
    # Randomly flip left-right
    if tf.random.uniform(()) > 0.25:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)

    # Randomly flip up-down
    if tf.random.uniform(()) > 0.25:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)

    # Apply color transformation (RGB to grayscale)
    if tf.random.uniform(()) > 0.25:
        image = tf.image.rgb_to_grayscale(image)

    # Adjust JPEG quality (simulating blur)
    if tf.random.uniform(()) > 0.25:
        image = tf.numpy_function(gaussian_blur, [image], tf.float32)
        image = tf.convert_to_tensor(image, dtype=tf.float32)

    return image, mask

In [18]:
# Apply data augmentation to your dataset
augmented_dataset = dataset.map(data_augmentation, num_parallel_calls=AUTOTUNE)

In [19]:
augmented_elems = list(augmented_dataset.as_numpy_iterator())

In [31]:
len(augmented_elems)

1404

In [33]:
combined_dataset = dataset.concatenate(augmented_dataset)

In [34]:
combined_elems = list(combined_dataset.as_numpy_iterator())

In [35]:
len(combined_elems)

2808